In [ ]:
import os
import sys
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.distributions import Normal
from src.strategy.buffer import Buffer
%matplotlib inline

In [ ]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
from src.strategy.model import Model
from src.strategy.agent import Agent
from src.strategy.environment import Environment
from src.utils import get_config, read_file
config = get_config.read_yaml()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = len(config['data']['symbols']) * config['data']['num_features']
gamma = config['hyperparameters']['gamma']
gae_lambda = config['hyperparameters']['gae_lambda']
clip_epsilon = config['hyperparameters']['clip_epsilon']
value_loss_coef = config['hyperparameters']['value_loss_coef']
entropy_loss_coef = config['hyperparameters']['entropy_loss_coef']
batch_size = config['hyperparameters']['batch_size']
epochs = config['hyperparameters']['num_epochs']
lr = config['hyperparameters']['learning_rate']
window_size = config['hyperparameters']['window_size']
print(f'input_dim: {input_dim}')
print(f'gamma: {gamma}')
print(f'gae_lambda: {gae_lambda}')
print(f'clip_epsilon: {clip_epsilon}')
print(f'value_loss_coef: {value_loss_coef}')
print(f'entropy_loss_coef: {entropy_loss_coef}')
print(f'batch_size: {batch_size}')
print(f'epochs: {epochs}')
print(f'lr: {lr}')
print(f'window_size: {window_size}')
print(f'device: {device}')

In [ ]:
data = read_file.read_merged_training_data()
data

In [ ]:
model = Model(input_dim).to(device)
model

In [ ]:
h_0, c_0 = model.init_hidden_state(batch_size, device)
h_0.shape

In [ ]:
x = []
for _ in range(batch_size):
    y = []
    for i in range(window_size):
        y.append(data.loc[data.index[i + _]])
    x.append(y)
x_t = torch.tensor(x).float().to(device)
print(x_t)
print(x_t.shape)

In [ ]:
action, log_prob, entropy, value, hidden_state_out, buffer = model.get_action_and_value(x_t, (h_0, c_0))
# print(f'dist: {dist}')
# print(f'value: {value}')
# print(f'hidden state: {hidden_state}')
buffer.display()

In [ ]:
all_means = mean.detach().cpu().numpy().flatten()
all_stds = std.detach().cpu().numpy().flatten()

In [ ]:
for _ in range(10):
    mea = all_means[_]
    st = all_stds[_]
    dis = Normal(mea, st)
    x_values = torch.linspace(mea - 4 * st, mea + 4 * st, steps=200)
    pdf_values = torch.exp(dis.log_prob(x_values))
    plt.figure(figsize=(8, 4))
    plt.plot(x_values.detach().numpy(), pdf_values.detach().numpy(), label=f'Normal(μ={mea.item():.2f}, σ={st.item():.2f})')
    plt.title('Probability Density Function (PDF) of a Normal Distribution')
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
action, log_prob, entropy, value, hidden_state = model.get_action_and_value(x_t, hidden_state=(h_0, c_0))
print(f'action: {action}')
print(f'log_prob: {log_prob}')
print(f'entropy: {entropy}')
print(f'value: {value}')
print(f'hidden state: {hidden_state}')

In [ ]:
action[0]

In [ ]:
action[0][9] = abs(action[0][9])
long_action = action[0][action[0] >= 0]
print(long_action)
short_action = action[0][action[0] < 0]
print(short_action)
all_action = long_action